<a href="https://colab.research.google.com/github/blindauth/labelshiftexperiments/blob/master/notebooks/KaggleDiabeticRetinopathy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://zenodo.org/record/3459399/files/kaggle_dr.txt.gz?download=1 -O kaggle_dr.txt.gz

--2019-11-07 19:43:11--  https://zenodo.org/record/3459399/files/kaggle_dr.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106643 (104K) [application/octet-stream]
Saving to: ‘kaggle_dr.txt.gz’

kaggle_dr.txt.gz    100%[===================>] 104.14K   289KB/s    in 0.4s    

2019-11-07 19:43:12 (289 KB/s) - ‘kaggle_dr.txt.gz’ saved [106643/106643]



In [ ]:
import gzip
import numpy as np

orig_kaggle_onehot_labels = [] #5 classes
orig_kaggle_predlogits = []
for row in gzip.open("kaggle_dr.txt.gz"):
    _, kaggle_label, logits = row.decode("utf-8").rstrip().split("\t")
    kaggle_label = int(kaggle_label)
    logits = [float(x) for x in logits.split(",")]
    labelsvec = np.zeros(5)
    labelsvec[kaggle_label] = 1.0
    orig_kaggle_onehot_labels.append(labelsvec)
    orig_kaggle_predlogits.append(logits)
orig_kaggle_onehot_labels = np.array(orig_kaggle_onehot_labels)
orig_kaggle_predlogits = np.array(orig_kaggle_predlogits)

In [ ]:
from sklearn.metrics import roc_auc_score
from scipy.special import softmax

print(roc_auc_score(y_true=1-orig_kaggle_onehot_labels[:,0],
                    y_score=1-softmax(orig_kaggle_predlogits, axis=-1)[:,0]))

0.8945151902722942


In [10]:
![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd /content/abstention
!git pull
!pip uninstall -y abstention
!pip install .
%cd ..

/content/abstention
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/blindauth/abstention
   6537110..294d173  master     -> origin/master
Updating 6537110..294d173
Fast-forward
 MNIST.ipynb | 175 ++++++++++++++++++++++++++++++++++++------------------------
 1 file changed, 105 insertions(+), 70 deletions(-)
Uninstalling abstention-0.1.3.0:
  Successfully uninstalled abstention-0.1.3.0
Processing /content/abstention
  Created wheel for abstention: filename=abstention-0.1.3.0-cp36-none-any.whl size=47481 sha256=f1268b313e6dad6997955eea24140427b78efc21e747c9b70e59a4ad4559211d
  Stored in directory: /tmp/pip-ephem-wheel-cache-fj3gwucs/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention


/content


In [11]:
![[ -e label_shift_experiments ]] || git clone https://github.com/blindauth/labelshiftexperiments
%cd /content/labelshiftexperiments
!git pull
!pip uninstall -y labelshiftexperiments
!pip install .
%cd ..

fatal: destination path 'labelshiftexperiments' already exists and is not an empty directory.
/content/labelshiftexperiments
remote: Enumerating objects: 2, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 8 (delta 2), reused 2 (delta 2), pack-reused 6
Unpacking objects: 100% (8/8), done.
From https://github.com/blindauth/labelshiftexperiments
   50c92a3..a455362  master     -> origin/master
Updating 50c92a3..a455362
Fast-forward
 notebooks/CIFAR10.ipynb                   | 132 ++++++++++++-----------
 notebooks/KaggleDiabeticRetinopathy.ipynb | 173 ++++++++++++------------------
 2 files changed, 139 insertions(+), 166 deletions(-)
Uninstalling labelshiftexperiments-0.1.0.0:
  Successfully uninstalled labelshiftexperiments-0.1.0.0
Processing /content/labelshiftexperiments
  Created wheel for labelshiftexperiments: filename=labelshiftexperiments-0.1.0.0-cp36-none-any.whl size=6624 sha256=069648536d2acb764d3972e34e2d18aeca98e4245e77ca382673ef215e0b64d2
  Stored in directo

/content


In [ ]:
#do calibration and domain adaptation
from collections import OrderedDict
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
from abstention.calibration import (
    TempScaling, NoBiasVectorScaling, VectorScaling, CrossValidatedBCTS, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
                                    BBSEImbalanceAdapter,
                                    RLLSImbalanceAdapter)

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('NBVS', NoBiasVectorScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions='all')),
    ('VS', VectorScaling(verbose=False))
])

adaptname_to_adapter = OrderedDict([
    ('empriorsfromlabels', EMImbalanceAdapter(estimate_priors_from_valid_labels=True)),
    ('empriorsfrompreds', EMImbalanceAdapter()),
    ('bbse-hard', BBSEImbalanceAdapter(soft=False)),
    ('bbse-soft', BBSEImbalanceAdapter(soft=True)),
    ('rlls-hard', RLLSImbalanceAdapter(soft=False)),
    ('rlls-soft', RLLSImbalanceAdapter(soft=True))
])
    
adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'NBVS'),
    ('bbse-soft', 'BCTS'),
    ('bbse-soft', 'VS'),
    ('bbse-soft', 'best-ece'),
    ('bbse-soft', 'best-jsdiv'),
    ('bbse-soft', 'best-nll'),

    ('rlls-hard', 'None'),
    ('rlls-soft', 'None'),
    ('rlls-soft', 'TS'),
    ('rlls-soft', 'NBVS'),
    ('rlls-soft', 'BCTS'),
    ('rlls-soft', 'VS'),
    ('rlls-soft', 'best-ece'),
    ('rlls-soft', 'best-jsdiv'),
    ('rlls-soft', 'best-nll'),

    ('empriorsfrompreds', 'None'),
    ('empriorsfrompreds', 'TS'),
    ('empriorsfrompreds', 'NBVS'),
    ('empriorsfrompreds', 'BCTS'),
    ('empriorsfrompreds', 'VS'),
    ('empriorsfromlabels', 'None'),
    ('empriorsfromlabels', 'TS'),
    ('empriorsfromlabels', 'NBVS'),
    ('empriorsfromlabels', 'BCTS'),
    ('empriorsfromlabels', 'VS'),
    ('empriorsfrompreds', 'best-ece'),
    ('empriorsfrompreds', 'best-jsdiv'),
    ('empriorsfrompreds', 'best-nll')
]
    

In [ ]:
len(orig_kaggle_onehot_labels)

3514

In [ ]:
np.mean(orig_kaggle_onehot_labels, axis=0)

array([0.7310757 , 0.07057484, 0.15253273, 0.02447353, 0.0213432 ])

In [ ]:
from collections import defaultdict
import numpy as np
import sys
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
from abstention.calibration import (
    TempScaling, #CrossValidatedBCTS,
    ConfusionMatrix, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
     BBSEImbalanceAdapter, ShiftWeightFromImbalanceAdapter)
import glob
import numpy as np
from collections import defaultdict, OrderedDict
import scipy

import labelshiftexperiments
reload(labelshiftexperiments)
reload(labelshiftexperiments.cifarandmnist)
from labelshiftexperiments import cifarandmnist

metric_to_adaptname_to_perfs = defaultdict(lambda: defaultdict(list))

num_trials = 100
tweakone_alphas_and_samplesize = [(0.5,500), (0.9,500),
                                  (0.5, 1000), (0.9, 1000),
                                  (0.5,1500), (0.9,1500)]

tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals = (
        defaultdict(
            lambda: defaultdict(
                     lambda: defaultdict(
                              lambda: defaultdict(list)))))
tweakone_alpha_to_samplesize_to_baselineperf = defaultdict(lambda: defaultdict(list))
metric_to_samplesize_to_calibname_to_unshiftedvals = defaultdict(
    lambda: defaultdict(lambda: defaultdict(list)))
samplesizesseen = set()

orig_class_priors = np.mean(orig_kaggle_onehot_labels, axis=0)

for (alpha, samplesize) in tweakone_alphas_and_samplesize:
    for trail_num in range(num_trials):
        rng = np.random.RandomState(trail_num*10)
        #sample a split of pseudovalid/pseudotest indices
        indices = rng.choice(np.arange(len(orig_kaggle_onehot_labels)),
                             size=len(orig_kaggle_onehot_labels),
                             replace=False)
        pseudovalid_indices = indices[:samplesize]
        #calibrateonme_indices = indices[samplesize:2*samplesize]
        pseudotest_indices = indices[samplesize:]
        assert ((len(pseudovalid_indices)+len(pseudotest_indices))
                ==len(orig_kaggle_onehot_labels))
        
        pseudovalid_preacts = orig_kaggle_predlogits[pseudovalid_indices]
        pseudotest_preacts = orig_kaggle_predlogits[pseudotest_indices]
        pseudovalid_labels = orig_kaggle_onehot_labels[pseudovalid_indices]
        pseudotest_labels = orig_kaggle_onehot_labels[pseudotest_indices]
        
        draw_test_indices = cifarandmnist.get_func_to_draw_label_proportions(
                              test_labels=pseudotest_labels)
        
        #perform calibration
        (calibname_to_calibfunc,
         calibname_to_calibpseudovalidpreds) = (
                    cifarandmnist.run_calibmethods(
                       valid_preacts=pseudovalid_preacts,
                       valid_labels=pseudovalid_labels,
                       test_preacts=pseudotest_preacts,
                       test_labels=pseudotest_labels,
                       calibname_to_calibfactory=
                        calibname_to_calibfactory,
                       samplesize=samplesize,
                       samplesizesseen=samplesizesseen,
                       metric_to_samplesize_to_calibname_to_unshiftedvals=
                        metric_to_samplesize_to_calibname_to_unshiftedvals))
        
        #note the calibration method that did the best according to each metric,
        # and save it
        for metricname in metric_to_samplesize_to_calibname_to_unshiftedvals:
            calibname_to_unshiftedvals = metric_to_samplesize_to_calibname_to_unshiftedvals[metricname][samplesize]
            best_calibname = min(list(calibname_to_unshiftedvals.keys()),
                                 key=lambda x: calibname_to_unshiftedvals[x][-1])
            calibname_to_calibfunc['best-'+metricname] = calibname_to_calibfunc[best_calibname]
            calibname_to_calibpseudovalidpreds['best-'+metricname] = calibname_to_calibpseudovalidpreds[best_calibname]
        
        #perform the knockout shift
        altered_class_priors = np.zeros(5)
        altered_class_priors[0] = alpha
        altered_class_priors[1:] = (1-alpha)*orig_class_priors[1:]
        
        pseudotest_indices = draw_test_indices(
                          total_to_return=samplesize,                    
                          label_proportions=altered_class_priors,        
                          rng=rng)
        shifted_pseudotest_labels = pseudotest_labels[pseudotest_indices]
        shifted_pseudotest_preacts = pseudotest_preacts[pseudotest_indices]
        
        calibname_to_calibshiftedtestpreds = {}
        for (calibname, calibfunc) in calibname_to_calibfunc.items():
            calibname_to_calibshiftedtestpreds[calibname] =(
                calibfunc(shifted_pseudotest_preacts))
        
        shifted_pseudotest_baseline_perf = np.mean(
            np.argmax(shifted_pseudotest_preacts, axis=-1)
            ==np.argmax(shifted_pseudotest_labels, axis=-1))
        tweakone_alpha_to_samplesize_to_baselineperf[alpha][samplesize].append(
            shifted_pseudotest_baseline_perf)
        ideal_shift_weights = (np.mean(shifted_pseudotest_labels,axis=0)/
                               np.mean(pseudovalid_labels,axis=0))
        true_shifted_priors = np.mean(shifted_pseudotest_labels, axis=0)
        
        for adapter_name,calib_name in adaptncalib_pairs:
            calib_shifted_pseudotest_preds =\
              calibname_to_calibshiftedtestpreds[calib_name]
            calib_pseudovalid_preds =\
              calibname_to_calibpseudovalidpreds[calib_name]
            imbalance_adapter = adaptname_to_adapter[adapter_name]
            imbalance_adapter_func = imbalance_adapter(
                valid_labels=pseudovalid_labels,                       
                tofit_initial_posterior_probs=calib_shifted_pseudotest_preds, 
                valid_posterior_probs=calib_pseudovalid_preds)
            adapted_shifted_pseudotest_preds = imbalance_adapter_func(        
                        calib_shifted_pseudotest_preds)
            shift_weights = imbalance_adapter_func.multipliers          

            unnormed_estimshiftedpriors = np.mean(                     
                pseudovalid_labels, axis=0)*shift_weights              
            estim_shifted_priors = (unnormed_estimshiftedpriors/        
                np.sum(unnormed_estimshiftedpriors*shift_weights))
            
            #compute metrics for quality of shift
            adapted_shifted_pseudotest_perf = np.mean(
              np.argmax(adapted_shifted_pseudotest_preds, axis=-1)
              ==np.argmax(shifted_pseudotest_labels, axis=-1))
            delta_from_baseline = (adapted_shifted_pseudotest_perf-
                                   shifted_pseudotest_baseline_perf)
            #expected value of mse weights; weighted by the class       
            # proportions in the test set                               
            tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[       
                alpha][samplesize][adapter_name+":"+calib_name][        
                'mseweights_testsetprop'].append(                       
                  np.sum(true_shifted_priors*(                          
                    np.square(ideal_shift_weights-shift_weights))))      
            #mse weights but each class weighted evenly                 
            tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[       
                alpha][samplesize][adapter_name+":"+calib_name][        
                'mseweights_even'].append(                              
                  np.mean(np.square(                                    
                    ideal_shift_weights-shift_weights)))
            tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                alpha][samplesize][adapter_name+":"+calib_name][
                'jsdiv'].append(
                  scipy.spatial.distance.jensenshannon(
                      p=true_shifted_priors, q=estim_shifted_priors))
            tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                alpha][samplesize][adapter_name+":"+calib_name][
                'delta_perf'].append(delta_from_baseline)
        
    if (samplesize not in samplesizesseen):
        print("Calibration stats")
        for metric in ['ece', 'nll', 'jsdiv']:
            print("Metric",metric)
            for calibname in calibname_to_calibfactory:
                print(calibname, np.mean(                     
                       metric_to_samplesize_to_calibname_to_unshiftedvals[
                        metric][samplesize][calibname]))  
        samplesizesseen.add(samplesize)

    print("On alpha",alpha,"sample size", samplesize)
    for metric_name in ['delta_perf', 'jsdiv']: 
        print("Metric",metric_name)
        for adapter_name,calib_name in adaptncalib_pairs:
            adaptncalib_name = adapter_name+":"+calib_name
            n = len(tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                  alpha][samplesize][adaptncalib_name][metric_name])

            print(adaptncalib_name, np.mean(
             tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
               alpha][samplesize][adaptncalib_name][metric_name]), "+/-",
             (1.0/np.sqrt(n))*np.std(
             tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
               alpha][samplesize][adaptncalib_name][metric_name],
               ddof=1))           
            sys.stdout.flush()

Calibration stats
Metric ece
None 8.733662753798031
TS 3.649580871054098
NBVS 2.1297461941216516
BCTS 2.2547945348797995
VS 2.3231526172283683
Metric nll
None 0.6402319025011183
TS 0.5711691961104738
NBVS 0.5433706417895835
BCTS 0.5144445766964391
VS 0.5180747116069775
Metric jsdiv
None 0.07661477273081052
TS 0.09092214616308848
NBVS 0.045375127921321035
BCTS 0.02565906567873681
VS 0.025921436855378746
On alpha 0.5 sample size 500
Metric delta_perf
bbse-hard:None -0.026600000000000026 +/- 0.01381732187648474
bbse-soft:None 0.003419999999999965 +/- 0.007343645366759433
bbse-soft:TS 0.003359999999999972 +/- 0.004355595959408558
bbse-soft:NBVS 0.009099999999999962 +/- 0.0076981632336367845
bbse-soft:BCTS 0.015599999999999961 +/- 0.009696859847862652
bbse-soft:VS 0.02517999999999996 +/- 0.002664710646258393
bbse-soft:best-ece 0.020479999999999957 +/- 0.0029341761874485126
bbse-soft:best-jsdiv 0.02527999999999996 +/- 0.002753072847022049
bbse-soft:best-nll 0.015459999999999963 +/- 0.0096928

In [ ]:
import json
import os
file_out = "kaggledr_label_shift_adaptation_results.json"
dict_to_write = {
    "tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "tweakone_alpha_to_samplesize_to_baselineperf":
     tweakone_alpha_to_samplesize_to_baselineperf,
    "metric_to_samplesize_to_calibname_to_unshiftedvals":
     metric_to_samplesize_to_calibname_to_unshiftedvals
}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0

In [1]:
import gzip
import json
loaded_dicts = json.loads(gzip.open("kaggledr_label_shift_adaptation_results.json.gz").read())
tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
tweakone_alpha_to_samplesize_to_baselineperf =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_baselineperf']
metric_to_samplesize_to_calibname_to_unshiftedvals =\
    loaded_dicts['metric_to_samplesize_to_calibname_to_unshiftedvals']

In [2]:
from importlib import reload
import labelshiftexperiments
reload(labelshiftexperiments)
import abstention.figure_making_utils
reload(abstention.figure_making_utils)
import labelshiftexperiments.maketable
reload (labelshiftexperiments.maketable)
from labelshiftexperiments.maketable import render_calibration_table
import numpy as np

metricname_to_nicename = {'nll': 'NLL', 'jsdiv': 'jsdiv', 'ece': 'ECE'}
calibname_to_nicename = {'None': "None", "TS": "TS",
                         "VS":"VS", "NBVS": "NBVS", "BCTS": "BCTS"}
  
from scipy.stats import norm
N = len(tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals[
        '0.5']['500']['empriorsfrompreds:TS']['jsdiv'])
#Using the normal approximation at N=100;
# variance from https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test
#Note that T = ((N+1)*N/2 - W)/2            
ustat_threshold = ((N*(N+1))/2 - norm.ppf(0.99)*np.sqrt(N*(N+1)*(2*N+1)/6.0))/2.0
  
print(render_calibration_table(
        metric_to_samplesize_to_calibname_to_unshiftedvals=
          metric_to_samplesize_to_calibname_to_unshiftedvals,
        #threshold of 8 comes from table https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
        #for one-tailed alpha=0.025 and n=10            
        ustat_threshold=ustat_threshold,
        metrics_in_table=['nll', 'ece'],
        samplesizes_in_table=['500', '1000', '1500'],
        calibnames_in_table=['None', 'TS', 'NBVS', 'BCTS', 'VS'],
        metricname_to_nicename=metricname_to_nicename,
        calibname_to_nicename=calibname_to_nicename,
        caption="KaggleDR Calibration metric differences", label="kaggledr_calibrationcomparison",
        applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c c c | c c c }
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{NLL} & \multicolumn{3}{| c}{ECE}\\
    \cline{2-7}
    & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500\\
    \hline
    None & 0.641; 4.0 & 0.64; 4.0 & 0.638; 4.0 & 8.738; 4.0 & 8.746; 4.0 & 8.745; 4.0\\
    TS & 0.57; 3.0 & 0.57; 3.0 & 0.568; 3.0 & 3.636; 3.0 & 3.749; 3.0 & 3.872; 3.0\\
    NBVS & 0.542; 2.0 & 0.54; 2.0 & 0.539; 2.0 & \textbf{1.956; 0.0} & \textbf{2.012; 1.0} & \textbf{2.047; 1.0}\\
    BCTS & \textbf{0.513; 0.0} & \textbf{0.511; 1.0} & 0.509; 1.0 & 2.157; 1.0 & \textbf{2.135; 1.0} & \textbf{2.08; 1.0}\\
    VS & 0.518; 1.0 & \textbf{0.512; 0.0} & 0.509; 0.0 & 2.187; 1.0 & \textbf{2.006; 1.0} & \textbf{2.014; 1.0}\\
  \end{tabular}}
  \caption{KaggleDR Calibration metric differences}
  \label{tab:kaggledr_calibrationcomparison}
\end{table

In [3]:
from labelshiftexperiments.maketable import render_adaptation_table
from collections import OrderedDict
    
metric_to_largerisbetter = {'perf':True, 'jsdiv': False}
samplesizes_in_table = ['500', '1000', '1500']
adaptname_to_nicename = {'empriorsfrompreds': 'EM',
                         'bbse-soft': 'BBSL-soft',
                         'bbse-hard': 'BBSL-hard',
                         'rlls-soft': 'RLLS-soft',
                         'rlls-hard': 'RLLS-hard'}
calibname_to_nicename = {'None': 'None',
                           'TS': 'TS',
                           'NBVS': 'NBVS',
                           'BCTS': 'BCTS',
                           'VS': 'VS',
                           'best-nll':'Best NLL',
                           'best-jsdiv':'Best JS Div',
                           'best-ece':'Best ECE'}
tweakone_alphas_in_table = ['0.5', '0.9']

methodgroups_all = OrderedDict([('all', [
                      'empriorsfrompreds:None', 'empriorsfrompreds:TS', 'empriorsfrompreds:NBVS', 'empriorsfrompreds:BCTS', 'empriorsfrompreds:VS',
                      'bbse-hard:None', 'bbse-soft:None', 'bbse-soft:TS', 'bbse-soft:NBVS', 'bbse-soft:BCTS', 'bbse-soft:VS',
                      'rlls-hard:None', 'rlls-soft:None', 'rlls-soft:TS', 'rlls-soft:NBVS', 'rlls-soft:BCTS', 'rlls-soft:VS'
                      ])])
methodgroups_em = OrderedDict([('em', ['empriorsfrompreds:None', 'empriorsfrompreds:TS',
                                       'empriorsfrompreds:NBVS', 'empriorsfrompreds:BCTS', 'empriorsfrompreds:VS'])])
methodgroups_bbserlls = OrderedDict([('bbse',
                                      ['bbse-soft:None', 'bbse-soft:TS',
                                       'bbse-soft:NBVS', 'bbse-soft:BCTS',
                                       'bbse-soft:VS']),
                                     ('rlls',
                                      ['rlls-soft:None', 'rlls-soft:TS',
                                       'rlls-soft:NBVS', 'rlls-soft:BCTS',
                                       'rlls-soft:VS'])])

#Accuracy improvement
methodgroups = methodgroups_em
print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=
      tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_perf',
    largerisbetter=True,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption=("KaggleDR, EM, Evaluation Metric: $\\Delta$\\%Accuracy"),
    label="kaggledr_em_deltaacc",
    applyunderline=False,
    symbol="\\rho"))

#JSDiv and MSE with all
methodgroups = methodgroups_all
for metric, nicemetricname, decimals in [('jsdiv', 'JS Divergence',3),
                                         ('mseweights_even', 'MSE Across Classes',5)]:
  print(render_adaptation_table(
      alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=
        tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
      ustat_threshold=ustat_threshold,
      valmultiplier=1.0,
      adaptname_to_nicename=adaptname_to_nicename,
      calibname_to_nicename=calibname_to_nicename,
      methodgroups=methodgroups,
      metric=metric,
      largerisbetter=False,
      alphas_in_table=tweakone_alphas_in_table,
      samplesizes_in_table=samplesizes_in_table,
      caption="KaggleDR ALL Evaluation Metric: "+nicemetricname,
      label="kaggledr_all_"+metric,
      applyunderline=False,
      symbol="\\rho",
      decimals=decimals))

#Shift estimates from BBSE vs. RLLS vs. EM
methodgroups_bycalib = OrderedDict([
  ('None', ['empriorsfrompreds:None', 'bbse-hard:None',
            'bbse-soft:None', 'rlls-hard:None', 'rlls-soft:None']),
  ('TS', ['empriorsfrompreds:TS', 'bbse-soft:TS', 'rlls-soft:TS']),
  ('NBVS', ['empriorsfrompreds:NBVS', 'bbse-soft:NBVS', 'rlls-soft:NBVS']),
  ('BCTS', ['empriorsfrompreds:BCTS', 'bbse-soft:BCTS', 'rlls-soft:BCTS']),
  ('VS', ['empriorsfrompreds:VS', 'bbse-soft:VS', 'rlls-soft:VS'])])
methodgroups_bycalib_fordeltaacc = OrderedDict([
  ('TS', ['empriorsfrompreds:TS', 'bbse-soft:TS', 'rlls-soft:TS']),
  ('NBVS', ['empriorsfrompreds:NBVS', 'bbse-soft:NBVS', 'rlls-soft:NBVS']),
  ('BCTS', ['empriorsfrompreds:BCTS', 'bbse-soft:BCTS', 'rlls-soft:BCTS']),
  ('VS', ['empriorsfrompreds:VS', 'bbse-soft:VS', 'rlls-soft:VS'])])

for (metric,nicemetricname,
     valmultiplier,largerisbetter,
     adaptname_to_nicename_touse,
     methodgroups_touse,
     decimals) in [('jsdiv', 'JS Divergence', 1.0, False, adaptname_to_nicename, methodgroups_bycalib, 3),
                   ('mseweights_even', 'MSE Across Classes', 1.0, False, adaptname_to_nicename, methodgroups_bycalib, 5)]:
  print(render_adaptation_table(
      alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
      ustat_threshold=ustat_threshold,
      valmultiplier=valmultiplier,
      adaptname_to_nicename=adaptname_to_nicename_touse,
      calibname_to_nicename=calibname_to_nicename,
      methodgroups=methodgroups_touse,
      metric=metric,
      largerisbetter=largerisbetter,
      alphas_in_table=tweakone_alphas_in_table,
      samplesizes_in_table=samplesizes_in_table,
      caption="KaggleDR, EM vs BBSL vs RLLS, Evaluation Metric: "+nicemetricname,
      label="kaggledr_emvsbbsevsrlls_"+metric,
      applyunderline=False,
      symbol="\\rho",
      decimals=decimals))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\rho=0.5$} & \multicolumn{3}{| c}{$\rho=0.9$}\\ 
    \cline{3-8}
    & & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500\\
    \hline
    \hline
    EM & None & 2.0; 3.0 & 2.0; 4.0 & 2.2; 4.0 & 1.4; 4.0 & 1.5; 4.0 & 1.567; 4.0\\
    EM & TS & 2.2; 3.0 & 2.3; 3.0 & 2.667; 3.0 & 1.6; 3.0 & 2.0; 3.0 & 2.133; 3.0\\
    EM & NBVS & 3.5; 2.0 & 4.1; 2.0 & 4.233; 2.0 & 2.2; 2.0 & 2.4; 2.0 & 2.467; 2.0\\
    EM & BCTS & \textbf{3.8; 1.0} & \textbf{4.65; 1.0} & \textbf{4.633; 1.0} & \textbf{3.6; 0.0} & \textbf{3.6; 0.0} & \textbf{3.733; 0.0}\\
    EM & VS & \textbf{3.8; 1.0} & \textbf{4.4; 1.0} & \textbf{4.633; 1.0} & \textbf{3.5; 1.0} & \textbf{3.6; 1.0} & \textbf{3.733; 1.0}\

In [4]:
methodgroups = OrderedDict([
  ('em-calib', ['empriorsfrompreds:best-nll', 'empriorsfrompreds:best-ece'])])
print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_perf',
    largerisbetter=True,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="\\textbf{KaggleDR: NLL vs ECE, $\\Delta$\\%Accuracy.}",
    label="kaggledr_nllvsece_deltaperf",
    applyunderline=False,
    symbol="\\rho"))


methodgroups = OrderedDict([
  ('em-calib', ['empriorsfrompreds:best-nll', 'empriorsfrompreds:best-ece']),
  ('bbse-calib', ['bbse-soft:best-nll', 'bbse-soft:best-ece']),
  ('rlls-calib', ['rlls-soft:best-nll', 'rlls-soft:best-ece'])])
for metric, nicemetricname in [('jsdiv', 'JS Divergence'),
                               ('mseweights_even', 'MSE Across Classes')]:
  print(render_adaptation_table(
      alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
      ustat_threshold=ustat_threshold,
      valmultiplier=1,
      adaptname_to_nicename=adaptname_to_nicename,
      calibname_to_nicename=calibname_to_nicename,
      methodgroups=methodgroups,
      metric=metric,
      largerisbetter=False,
      alphas_in_table=tweakone_alphas_in_table,
      samplesizes_in_table=samplesizes_in_table,
      caption="\\textbf{KaggleDR: NLL vs ECE, "+nicemetricname+".}",
      label="kaggledr_nllvsece_"+metric,
      applyunderline=False,
      symbol="\\rho"))



\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\rho=0.5$} & \multicolumn{3}{| c}{$\rho=0.9$}\\ 
    \cline{3-8}
    & & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500\\
    \hline
    \hline
    EM & Best NLL & \textbf{3.8; 0.0} & \textbf{4.5; 0.0} & \textbf{4.6; 0.0} & \textbf{3.6; 0.0} & 3.6; 0.0 & \textbf{3.733; 0.0}\\
    EM & Best ECE & 3.6; 1.0 & 4.4; 1.0 & 4.3; 1.0 & 2.2; 1.0 & 3.6; 1.0 & 2.467; 1.0\\
  \end{tabular}}
  \caption{\textbf{KaggleDR: NLL vs ECE, $\Delta$\%Accuracy.}}
  \label{tab:kaggledr_nllvsece_deltaperf}
\end{table*}


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estim

In [5]:
adaptname_to_nicename = {'empriorsfrompreds': 'EM: source priors from preds',
                         'empriorsfromlabels': 'EM: source priors from labels'}

methodgroups = OrderedDict([
  ('emnone', ['empriorsfrompreds:None', 'empriorsfromlabels:None']),
  ('emts', ['empriorsfrompreds:TS', 'empriorsfromlabels:TS']),
  ('emnbvs', ['empriorsfrompreds:NBVS', 'empriorsfromlabels:NBVS']),
  ('embcts', ['empriorsfrompreds:BCTS', 'empriorsfromlabels:BCTS']),
  ('emvs', ['empriorsfrompreds:VS', 'empriorsfromlabels:VS'])])


print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_perf',
    largerisbetter=True,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="KaggleDR different EM source priors $\\Delta$\\%Accuracy",
    label="kaggledr_diffemsourcepriors_deltaperf",
    applyunderline=False,
    symbol="\\rho"))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="KaggleDR different EM source priors JSDiv",
    label="kaggledr_diffemsourcepriors_jsdiv",
    applyunderline=False,
    symbol="\\rho"))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\rho=0.5$} & \multicolumn{3}{| c}{$\rho=0.9$}\\ 
    \cline{3-8}
    & & $n$=500 & $n$=1000 & $n$=1500 & $n$=500 & $n$=1000 & $n$=1500\\
    \hline
    \hline
    EM: source priors from preds & None & \textbf{2.0; 0.0} & \textbf{2.0; 0.0} & \textbf{2.2; 0.0} & \textbf{1.4; 0.0} & \textbf{1.5; 0.0} & \textbf{1.567; 0.0}\\
    EM: source priors from labels & None & -3.4; 1.0 & -3.6; 1.0 & -3.367; 1.0 & 1.0; 1.0 & 1.0; 1.0 & 1.067; 1.0\\
    \hline
    \hline
    EM: source priors from preds & TS & \textbf{2.2; 0.0} & \textbf{2.3; 0.0} & \textbf{2.667; 0.0} & \textbf{1.6; 0.0} & \textbf{2.0; 0.0} & \textbf{2.133; 0.0}\\
    EM: source priors from labels & TS & -64.5; 1.0 & -65.1; 1.0 